In [1]:
import glob
import numpy as np
import operator
import time


In [2]:

d = 1792
K = 20
print("Started at %s" %time.ctime())

    


Started at Mon Apr 27 01:19:55 2020


In [3]:
#subset of files for faster performance
allfiles = glob.glob('sub-npz/*.npz')
X = []
file_name_mapping = {}
for i, file_name in enumerate(allfiles):
    file_name_mapping[i] = file_name
    X.append(np.loadtxt(file_name))
    
print("total files: {}".format(len(X)))

total files: 1743


In [4]:
#generate query set, and random weights for testing
Q = np.random.randint(low=0, high=len(X), size=5)
queryFiles = [file_name_mapping[i] for i in Q]
W = [0.2, 0.3, 0.2, 0.5, 0.8]
Q = [X[i] for i in Q]



In [5]:
#calculate delta_ijl
delt = []
for l in range(0, d):
    delt_l = []
    for j in range(0, len(Q)):
        delt_j = []
        for i in range(0, len(X)):
            delta_value = W[j]*abs(X[i][l]-Q[j][l])
            delt_j.append((i, delta_value))
        delt_l.append(delt_j)
    delt.append(delt_l)
    


In [6]:
#"union" of the index list
KS_l_prime = []
for l in range(0, d):
    counter = {}
    for j in range(0, len(Q)):
        delt[l][j].sort(key = operator.itemgetter(1))
        for id, dist in [delt[l][j][k] for k in range(0, K)]:
            if id in counter:
                val = counter[id]
                counter[id] = (val[0]+1, val[1]+dist)
            else:
                counter[id] = (1, dist)
    KS_l_prime.append(counter)


In [7]:
#id of k loweset distance in dimension l
KS_l = []
for l in range(0, d):
    st = sorted(KS_l_prime[l].items(), key=lambda x: (-x[1][0], x[1][1]))
    KS_l.append(set([st[k][0] for k in range(0, K)]))

print(KS_l[0])
print(KS_l[1])
        

{261, 524, 1433, 28, 416, 44, 1070, 952, 324, 1734, 1607, 1246, 735, 480, 1124, 1129, 112, 626, 373, 1401}
{129, 1411, 261, 1168, 1177, 1434, 793, 28, 1317, 44, 1200, 178, 698, 1342, 735, 881, 373, 1525, 1401, 763}


In [8]:
#binary for each X_i, 1 if in KS_l of that dimension, 0 otherwise
B_i = []
for i in range(0, len(X)):
    b_i = []
    B_i.append([1 if i in KS_l[l] else 0 for l in range(0, d)])




In [9]:
def q_bar_l(l):
    nominator = 0
    denominator = 0
    for j in range(0, len(Q)):
        nominator += W[j] * Q[j][l]
        denominator += W[j]
    return nominator/denominator

#multiquery distance for each item
MSD_i = []
for i in range(0, len(X)):
    nominator = 0
    denominator = 0
    for l in range(0, d):
        if B_i[i][l] != 0:
            nominator += abs(X[i][l]-q_bar_l(l))
            denominator += 1
    denominator = denominator ** 2
    MSD_i.append((i, nominator/denominator if denominator > 0 else float('inf')))


    


In [10]:
st = sorted(MSD_i, key=lambda x: x[1])
MQKNN = [st[k] for k in range(0, K)]


In [11]:
print("QuerySet")
print(queryFiles)

print("MQKNN")
print([file_name_mapping[ans[0]] for ans in MQKNN])
print("Endedat at %s" %time.ctime())

QuerySet
['sub-npz\\0_46_3.npz', 'sub-npz\\1_139_1.npz', 'sub-npz\\2_266_1.npz', 'sub-npz\\0_107_4.npz', 'sub-npz\\0_104_4.npz']
MQKNN
['sub-npz\\0_101_2.npz', 'sub-npz\\0_0_4.npz', 'sub-npz\\0_100_5.npz', 'sub-npz\\0_0_2.npz', 'sub-npz\\0_100_4.npz', 'sub-npz\\0_0_3.npz', 'sub-npz\\0_102_4.npz', 'sub-npz\\0_103_2.npz', 'sub-npz\\0_101_5.npz', 'sub-npz\\0_102_1.npz', 'sub-npz\\0_101_4.npz', 'sub-npz\\0_0_1.npz', 'sub-npz\\0_0_5.npz', 'sub-npz\\0_100_2.npz', 'sub-npz\\0_104_4.npz', 'sub-npz\\0_103_1.npz', 'sub-npz\\0_102_2.npz', 'sub-npz\\0_101_1.npz', 'sub-npz\\0_101_3.npz', 'sub-npz\\0_103_4.npz']
Endedat at Mon Apr 27 01:20:29 2020


In [12]:
W = [0.2, 0.3, 0.2, 0.5, 0.8]
# conclusion, to use annoy clustering to reduce the image set, then apply MQKNN on the remaining image to get top 5.